# Find bars nearby soccer stadiums located in Germany

In [ ]:
from arcgis.gis import GIS
from arcgis.features import FeatureSet
from geopedestrian.services import solve_walking, walk_to
from georapid.client import GeoRapidClient
from georapid.factory import EnvironmentClientFactory

In [ ]:
class LocalClient(GeoRapidClient):
    
    def __init__(self, url, auth_headers):
        pass
    
    @property
    def auth_headers(self):
        return None
    
    @property
    def url(self):
        return 'http://localhost:7071/api'

## Location of Olympiastadion, Berlin

In [ ]:
lat, lon = 52.514610, 13.239665

In [ ]:
client = LocalClient(None, None)

In [ ]:
walking_geojson = solve_walking(client, lat, lon)
walking_fset = FeatureSet.from_geojson(walking_geojson)
walking_fset.sdf

In [ ]:
gis = GIS()

In [ ]:
berlin_map = gis.map('Berlin, Germany')

In [ ]:
walking_fset.sdf.spatial.plot(berlin_map)
berlin_map

## Places Category Bars

In [ ]:
category = 13003
walking_result = walk_to(client, lat, lon, category)
places_geojson = walking_result['places']
places_fset = FeatureSet.from_geojson(places_geojson)
places_fset.sdf

In [ ]:
places_fset.sdf.spatial.plot(berlin_map)
berlin_map

In [ ]:
routes_geojson = walking_result['routes']
routes_fset = FeatureSet.from_geojson(routes_geojson)
routes_fset.sdf

In [ ]:
routes_fset.sdf.spatial.plot(berlin_map)
berlin_map